In [ ]:
## In this test, we will study how to make flat ntuple out of cohpidf as an input for GUNDAM for fit

import os
import sys

import numpy as np
import math
import uproot as uproot
import pickle
import pandas as pd
import awkward as ak

import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import ticker
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
from matplotlib import gridspec

# Add the head direcoty to sys.path
workspace_root = os.getcwd()  
sys.path.insert(0, workspace_root + "/../../../")
#sys.path.insert(0, "/cvmfs/larsoft.opensciencegrid.org/spack-packages/opt/spack/linux-almalinux9-x86_64_v2/gcc-12.2.0/root-6.28.06-jhpj2jsdlwoxbvpnwmxvzkntrxcgw5of/lib/root")

# import this repo's classes
import pyanalib.pandas_helpers as ph




In [ ]:
## 1. Open each df
venv_path = os.getenv("PATH")
input_df_path = "/exp/sbnd/data/users/sungbino/sbnd_samples/cafpyana_outputs/cohpi_df_mcweight_test.df"
with pd.HDFStore(input_df_path) as store:
    print(store.keys())

In [ ]:
cohpidf = pd.read_hdf(input_df_path, key='cohpi')
hdrdf = pd.read_hdf(input_df_path, key='hdr')
mcnuwgtdf = pd.read_hdf(input_df_path, key='mcnuwgt')

In [ ]:
## Collect POT and scale factor to the target POT
this_pot = sum(hdrdf.pot)
print(this_pot)
target_POT = 3.0e18
POT_scale = target_POT / this_pot
print(POT_scale)

In [ ]:
## Make a df that matches mcnudf to slc
matchdf = ph.multicol_merge(cohpidf.reset_index(), mcnuwgtdf.reset_index(),
                            left_on=[("entry", "",""), ("rec", "slc","tmatch", "idx")],
                            right_on=[("entry", "",""), ("rec.mc.nu..index", "","")], 
                            how="left") ## -- save all sllices

In [ ]:
matchdf.GENIEReWeight_SBN_v1_multisigma_NonRESBGvbarnCC1pi	

In [ ]:
matchdf.columns

In [ ]:
matchdf.columns = ['.'.join(str(part) for part in col if part) for col in matchdf.columns]

In [ ]:
matchdf

In [ ]:
for col in matchdf.columns:
    if col.startswith("GENIEReWeight"):
        series = matchdf[col]
        first_valid = series.dropna().iloc[0]
        fill_value = [1.0] * len(first_valid)
        matchdf[col] = matchdf[col].apply(lambda x: x if isinstance(x, list) else fill_value)

In [ ]:
matchdf

In [ ]:
mcnuwgtdf

In [ ]:
mcnuwgtdf.columns = ['.'.join(str(part) for part in col if part) for col in mcnuwgtdf.columns]

In [ ]:
mcnuwgtdf = mcnuwgtdf.reset_index()

In [ ]:
mcnuwgtdf

In [ ]:
for col in mcnuwgtdf.columns:
    if col.startswith("GENIEReWeight"):
        series = mcnuwgtdf[col]
        first_valid = series.dropna().iloc[0]
        fill_value = [1.0] * len(first_valid)
        mcnuwgtdf[col] = mcnuwgtdf[col].apply(lambda x: x if isinstance(x, list) else fill_value)

In [ ]:
## Make TTree of matchdf
#ak_array = ak.Array(matchdf.to_dict(orient="list"))


In [ ]:
with uproot.recreate("gundam_output_test.root") as f:
    f["matched"] = matchdf
    f["mcnu"] = mcnuwgtdf